In [1]:
!git clone https://github.com/teamght/mascotas-model-app
!mv mascotas-model-app/saved_model ./saved_model

Cloning into 'mascotas-model-app'...
remote: Enumerating objects: 84, done.
remote: Counting objects: 100% (84/84), done.
remote: Compressing objects: 100% (65/65), done.
remote: Total 84 (delta 39), reused 60 (delta 19), pack-reused 0
Unpacking objects: 100% (84/84), done.


In [2]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
import tensorflow as tf
import os
import numpy as np
import skimage as sk
from skimage import io
import matplotlib.pyplot as plt
import tensorflow.keras.backend as K                              # Size of the input images

# Loss definition.

alpha = 0.3
def triplet(y_true,y_pred):
    
    a = y_pred[0::3]
    p = y_pred[1::3]
    n = y_pred[2::3]
    
    ap = K.sum(K.square(a-p),-1)
    an = K.sum(K.square(a-n),-1)

    return K.sum(tf.nn.relu(ap - an + alpha))

def triplet_acc(y_true,y_pred):
    a = y_pred[0::3]
    p = y_pred[1::3]
    n = y_pred[2::3]
    
    ap = K.sum(K.square(a-p),-1)
    an = K.sum(K.square(a-n),-1)
    
    return K.less(ap+alpha,an)

# Model load

PATH_MODEL  = './saved_model'      # Path to the directory where the model will be stored
SIZE        = (224,224,3) 

print('Loading model...')

model = tf.keras.models.load_model(PATH_MODEL,
    custom_objects={'triplet':triplet,'triplet_acc':triplet_acc})

print('Done')


Loading model...
Done


In [3]:
def load_images(filenames):
    """
    Use scikit-image library to load the pictures from files to numpy array.
    """
    h,w,c = SIZE
    images = np.empty((len(filenames),h,w,c))
    for i,f in enumerate(filenames):
        images[i] = sk.io.imread(f)/255.0
    return images

def predict_generator(filenames, batch_size=32):
    """
    Prediction generator.
    """
    for i in range(0,len(filenames),batch_size):
        print(f'predict_generator: {i} de {len(filenames)}')
        images_batch = load_images(filenames[i:i+batch_size])
        yield images_batch

In [4]:
def distance(v1, v2):
  diff = np.square(v1-v2)
  dist = np.sum(diff)
  return dist

In [6]:
# load images
from google.colab import files
uploaded = files.upload()
uploaded_files = list(uploaded.keys())
uploaded_files.sort()
print("Uploaded files:", uploaded_files)
# predict
generator = predict_generator(uploaded_files, batch_size=len(uploaded_files))
predict = model.predict_generator(generator, steps=1)
# borro los archivos subidos (por si dsp subo con el mismo nombre)
for filename in uploaded_files:
  !rm {filename}
# distances
predictions_list = list(predict)
#print(predictions_list)
print("\nDISTANCIAS\n#############################")
for i, image_a in enumerate(uploaded_files):
  for j, image_b in enumerate(uploaded_files):
    if i < j:
      d = distance(predictions_list[i], predictions_list[j])
      #print(i, j)
      print(image_a, image_b)
      print(d)
      print("#############################")

Saving c.jpg to c.jpg
Saving a.jpeg to a.jpeg
Saving b.jpeg to b.jpeg
Uploaded files: ['a.jpeg', 'b.jpeg', 'c.jpg']
predict_generator: 0 de 3


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:2001: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  warnings.warn('`Model.predict_generator` is deprecated and '



DISTANCIAS
#############################
a.jpeg b.jpeg
0.9293704
#############################
a.jpeg c.jpg
1.2705088
#############################
b.jpeg c.jpg
0.2621542
#############################
